# テトラコリック相関係数（tetrachoric correlation）

**テトラコリック相関係数** （tetrachoric correlation coefficient, **四分相関係数** とも） は、二値変数同士の相関係数で、2つの二値変数の背後に連続変数が存在すると仮定して、それらの間の相関の強さを推定する。


:::{dropdown} 歴史的経緯 （from [Olsson (1979)](https://www.cambridge.org/core/journals/psychometrika/article/abs/maximum-likelihood-estimation-of-the-polychoric-correlation-coefficient/62B8AC8F4B0EC5ED7F5B5037A9EAC40D)）

Pearson (1901) で標準正規分布の累積分布

$$
\Phi(\tau_X, \tau_Y ; \rho)=\frac{1}{2 \pi\sqrt{1-\rho^2}}
\int_{-\infty}^{\tau_X} \int_{-\infty}^{\tau_Y} \exp \left[-\frac{x^2-2 \rho x y+y^2}{2\left(1-\rho^2\right)}\right] d x d y
$$

から$\rho$を解くことで求める方法が提案された。

[Hamdan (1970)](https://www.jstor.org/stable/2334955) は2x2表から$\rho$を最尤推定することとtetrachoric correlationが等しいことを示した。

[Brown & Benedetti (1977)](https://www.cambridge.org/core/journals/psychometrika/article/abs/on-the-mean-and-variance-of-the-tetrachoric-correlation-coefficient/029C2190E6A9B8D40EB7D2924112E2A4) はtetrachoric correlationが対応する真の相関に対してバイアスがあるが、セルの予想される頻度が5未満でない場合、バイアスは無視できることを示した。

:::

### 推定方法

2つの観測変数$X, Y \in \{0, 1\}$があるとし、その背後に正規分布に従う潜在変数$X^*, Y^*$

$$
\left(X^*, Y^*\right) \sim \mathcal{N}\left(\left[\begin{array}{l}
0 \\
0
\end{array}\right],\left[\begin{array}{ll}
1 & \rho \\
\rho & 1
\end{array}\right]\right)
$$

があり、それぞれの潜在変数がある閾値によって二値に変換されている、すなわち

$$
X = \mathbb{1}( X^* > \tau_{X}), \quad
Y = \mathbb{1}( Y^* > \tau_{Y})
$$

とする。

相関係数$\rho$ と 2変量の標準正規分布の累積分布関数$\Phi_2(\tau_i, \tau_j; \rho)$ による同時確率

$$
P(X=i, Y=j) = \Phi_2(\tau_i, \tau_j; \rho)
$$

を考えて、ここから尤度関数を作って最尤推定を行う。

観測値のクロス集計表を以下のように計算したとする。

|       |  $Y=0$   |  $Y=1$   | 合計  |
| :---: | :------: | :------: | :---- |
| $X=0$ | $n_{00}$ | $n_{01}$ | $n_0$ |
| $X=1$ | $n_{10}$ | $n_{11}$ | $n_1$ |
| 合計  | $n_{.0}$ | $n_{.1}$ | $n$   |


仮定

$$
\begin{aligned}
& X=1 \Leftrightarrow X^*>\tau_X \\
& Y=1 \Leftrightarrow Y^*>\tau_Y
\end{aligned}
$$

により、

$$
\begin{aligned}
& X=0 \Leftrightarrow X^* \leq \tau_X \\
& Y=0 \Leftrightarrow Y^* \leq \tau_Y
\end{aligned}
$$

なので、標準正規分布の分位点関数$\Phi^{-1}$を使うと

$$
\begin{gathered}
\tau_X=\Phi^{-1}(P(X=0))=\Phi^{-1}\left(\frac{n_{00}+n_{01}}{n}\right) \\
\tau_Y=\Phi^{-1}(P(Y=0))=\Phi^{-1}\left(\frac{n_{00}+n_{10}}{n}\right)
\end{gathered}
$$

:::{margin}

標準正規分布の累積分布関数（CDF）を $\Phi(z)$ とすると ：

$$
\Phi(z)=P(Z \leq z) \quad \text { where } Z \sim \mathcal{N}(0,1)
$$


その逆関数が $\Phi^{-1}(p)$、つまり：

$$
\Phi^{-1}(p)=z \quad \text { such that } \Phi(z)=p
$$


:::

$P_{ij} = P(X=i, Y=j)$とすると

$$
\begin{aligned}
& P_{00}=\Phi_2\left(\tau_X, \tau_Y ; \rho\right) \\
& P_{01}=\Phi\left(\tau_X\right)-\Phi_2\left(\tau_X, \tau_Y ; \rho\right) \\
& P_{10}=\Phi\left(\tau_Y\right)-\Phi_2\left(\tau_X, \tau_Y ; \rho\right) \\
& P_{11}=1-\Phi\left(\tau_X\right)-\Phi\left(\tau_Y\right)+\Phi_2\left(\tau_X, \tau_Y ; \rho\right)
\end{aligned}
$$


ここで：
-  $\boldsymbol{\Phi}:$ 標準正規分布のCDF
- $\Phi_2(u, v ; \rho): 2$ 変量正規分布のCDF（平均0、分散1、相関 $\rho$）

そして次の対数尤度を最大化して推定する

$$
\log L(\rho)=n_{00} \log P_{00}+n_{01} \log P_{01}+n_{10} \log P_{10}+n_{11} \log P_{11}
$$

In [9]:
from scipy.optimize import minimize_scalar

def tetrachoric_corr(x: np.ndarray, y: np.ndarray) -> float:
    n = len(x)
    n00 = np.sum((x == 0) & (y == 0))
    n01 = np.sum((x == 0) & (y == 1))
    n10 = np.sum((x == 1) & (y == 0))
    n11 = np.sum((x == 1) & (y == 1))
    
    px0 = (n00 + n01) / n
    py0 = (n00 + n10) / n
    
    tau_x = norm.ppf(px0)
    tau_y = norm.ppf(py0)

    def neg_log_likelihood(rho):
        cov = np.array([[1, rho], [rho, 1]])
        p00 = multivariate_normal.cdf([tau_x, tau_y], mean=[0, 0], cov=cov)
        p01 = norm.cdf(tau_x) - p00
        p10 = norm.cdf(tau_y) - p00
        p11 = 1 - norm.cdf(tau_x) - norm.cdf(tau_y) + p00

        probs = np.array([p00, p01, p10, p11])
        counts = np.array([n00, n01, n10, n11])

        assert np.all(probs >= 0)
        return -np.sum(counts * np.log(probs))

    result = minimize_scalar(neg_log_likelihood, bounds=(-0.999, 0.999), method='bounded')
    return result.x

In [11]:
# データ生成（潜在相関0.6）
from scipy.stats import norm
np.random.seed(1)
latent_x = np.random.normal(size=100)
latent_y = 0.6 * latent_x + np.random.normal(size=100)
X = (latent_x > 0.3).astype(int)
Y = (latent_y > 0.3).astype(int)

# 推定
rho_hat = tetrachoric_corr(X, Y)
print(f"推定されたテトラコリック相関: {rho_hat:.4f}")

推定されたテトラコリック相関: 0.4084


## 参考文献

:::{card} 関連文献

- [Pearson, K. (1900). Mathematical contributions to the theory of evolution.—VII. On the correlation of characters not quantitatively measurable. Philosophical Transactions of the Royal Society of London. Series A, Containing Papers of a Mathematical or Physical Character, 195(262-273), 1-47.](https://royalsocietypublishing.org/doi/10.1098/rsta.1900.0022)
- [Hamdan, M. A. (1970). The equivalence of tetrachoric and maximum likelihood estimates of P in 2× 2 tables.](https://www.jstor.org/stable/2334955)
- [Brown, M. B., & Benedetti, J. K. (1977). On the mean and variance of the tetrachoric correlation coefficient. Psychometrika, 42(3), 347-355.](https://www.cambridge.org/core/journals/psychometrika/article/abs/on-the-mean-and-variance-of-the-tetrachoric-correlation-coefficient/029C2190E6A9B8D40EB7D2924112E2A4)
- [Everitt, P. F. (1910). Tables of the tetrachoric functions for fourfold correlation tables. Biometrika, 7(4), 437-451.](https://www.jstor.org/stable/2345377?origin=JSTOR-pdf)
- [Castellan Jr, N. J. (1966). On the estimation of the tetrachoric correlation coefficient. Psychometrika, 31(1), 67-73.](https://www.cambridge.org/core/journals/psychometrika/article/abs/on-the-estimation-of-the-tetrachoric-correlation-coefficient/60F0218237B83F1B00B93234B8B354FC)

:::